# Setup/Imports

In [ ]:
import pickle
import keras
import tensorflow as tf
from keras import backend as K
import numpy as np
from helpers.data_generator import process_data, DataGenerator
from helpers.custom_losses import denorm_loss, hinge_mse_loss
from helpers.custom_losses import percent_correct_sign, baseline_MAE
from models.LSTMConv2D import get_model_lstm_conv2d, get_model_simple_lstm
from models.LSTMConv2D import get_model_linear_systems, get_model_conv2d
from utils.callbacks import CyclicLR, TensorBoardWrapper
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from time import strftime, localtime
import matplotlib
from matplotlib import pyplot as plt
import sys
import os
sys.path.append(os.path.abspath('../'))
from helpers.normalization import normalize, denormalize, renormalize
import tkinter as tk
from tkinter import filedialog
root = tk.Tk()
root.withdraw()

In [22]:
num_cores = 4
config = tf.ConfigProto(intra_op_parallelism_threads=num_cores,
                        inter_op_parallelism_threads=num_cores, 
                        allow_soft_placement=True,
                        device_count = {'CPU' : 1,
                                        'GPU' : 0})
session = tf.Session(config=config)
K.set_session(session)

In [23]:
%matplotlib inline
font={'family': 'Times New Roman',
      'size': 10}
plt.rc('font', **font)
matplotlib.rcParams['figure.facecolor'] = (1,1,1,1)

# Functions

In [45]:
def hinge_loss(delta_true, delta_pred):
    """Finds the percentage of the time the prediction has the right sign
    """
    return np.sum(np.maximum(np.sign(delta_true*delta_pred),0))/delta_true.size

def batch_hinge(model,generator,param_dict,sig):
    """Finds the percentage of the time the prediction has the right sign
    """    
    nbatches = len(generator)
    err = 0
    for ind in range(nbatches):
        inputs, targets = generator[ind]
        predictions = model.predict_on_batch(inputs)
        full_pred, full_true, delta_pred, delta_true, baseline = batch_denorm(inputs,targets,predictions,param_dict,sig)
        err += hinge_loss(delta_true,delta_pred)
    return err/nbatches    

def batch_denorm(inputs,targets,predictions,param_dict,sig):
    """Denormalizes and calculates deltas, prep for plotting and analysis
    """
    targets = targets['target_' + sig]
    baseline = inputs['input_' + sig][:,-1]
    if predict_deltas:
        full_pred = predictions + baseline
        full_true = targets + baseline
    else:
        full_pred = predictions
        full_true = targets
    denorm_baseline = denormalize(baseline, param_dict[sig])
    denorm_full_pred = denormalize(full_pred, param_dict[sig])
    denorm_full_true = denormalize(full_true, param_dict[sig])
    denorm_delta_pred = denorm_full_pred - denorm_baseline
    denorm_delta_true = denorm_full_true - denorm_baseline
    return denorm_full_pred, denorm_full_true, denorm_delta_pred, denorm_delta_true, denorm_baseline

def plot_batch(y_true, y_pred, baseline, psi, labels, axlabels,shots,times):
    batch_size = y_true.shape[0]
    ncols = 4
    nrows = int(batch_size/ncols)
    figsize = (20,10*batch_size/20)
    fig, ax = plt.subplots(nrows=nrows, ncols=ncols, figsize=figsize)
    ax = ax.flatten()
    # Plot predictions and true deltas
    for i in range(batch_size):
        ax[i].plot(psi,y_true[i], label=labels[0])
        ax[i].plot(psi,y_pred[i], label=labels[1])
        ax[i].plot(psi,baseline[i], label=labels[2])
        ax[i].title.set_text('Shot ' + str(int(shots[i])) + ', time ' + str(int(times[i])))
        ax[i].set_xlabel(axlabels[0])
        ax[i].set_ylabel(axlabels[1])
        ax[i].legend(loc=0)
    plt.tight_layout()
    return fig,ax

# Analysis

In [28]:
file_path = filedialog.askopenfilename(title='Saved Model File',filetypes = [("hdf5 files","*.h5"),("all files","*.*")])
model = keras.models.load_model(file_path, compile=False)
print('loaded model: ' + file_path.split('/')[-1])
# file_path = filedialog.askopenfilename(title='Config Pickle',filetypes = [("pickle files","*.pkl"),("all files","*.*")])
# with open(file_path, 'rb') as f:
#      analysis_params = pickle.load(f, encoding='latin1')
# print('loaded dict: ' + file_path.split('/')[-1])
# print('with parameters: ' + str(analysis_params.keys()))

loaded model: model-conv2d_profiles-temp-dens-rotation-press-itemp-ffprime_act-pinj-curr-tinj-gasA_targ-temp_norm-StandardScaler_profLB-1_actLB-8_activ-relu_nshots-10000_29Jul19-21-10.h5


In [30]:
model_type = 'conv2d'
input_profile_names = ['temp', 'dens', 'rotation', 'press', 'itemp', 'ffprime']
target_profile_names = ['temp']
actuator_names = ['pinj', 'curr', 'tinj', 'gasA']
predict_deltas = False
profile_lookback = 1
actuator_lookback = 8
lookahead = 3
profile_length = 65
std_activation = 'relu'
rawdata_path = '/home/fouriest/SCHOOL/Princeton/PPPL/final_data.pkl'
checkpt_dir = '/home/fouriest/SCHOOL/Princeton/PPPL/'
sig_names = input_profile_names + target_profile_names + actuator_names
normalization_method = 'StandardScaler'
window_length = 1
window_overlap = 0
sample_step = 4
uniform_normalization = True
train_frac = 0.8
val_frac = 0.2
nshots = 1000
mse_weight_vector = np.linspace(1, np.sqrt(10), profile_length)**2
hinge_weight = 50
batch_size = 64
epochs = 100
verbose = 1

traindata, valdata, param_dict = process_data(rawdata_path, sig_names, normalization_method,
                                              window_length, window_overlap, max(
                                                  profile_lookback, actuator_lookback),
                                              lookahead, sample_step, uniform_normalization,
                                              train_frac, val_frac, nshots,verbose=1)

Loading
Signals: curr, dens, ffprime, gasA, itemp, pinj, press, rotation, temp, tinj, time, shotnum


Gathering:   2%|1         | 16/1000 [00:00<00:06, 153.88it/s]

Number of useable shots:  8683
Number of shots used:  1000
Total number of timesteps:  80431


Splitting: 100%|##########| 12/12 [00:00<00:00, 92.84it/s]

Total number of samples:  17746
Number of training samples:  14196
Number of validation samples:  3550


In [31]:
batch_size = 20
train_generator = DataGenerator(traindata, batch_size, input_profile_names,
                                actuator_names, target_profile_names,
                                profile_lookback, actuator_lookback, lookahead,
                                predict_deltas)
val_generator = DataGenerator(valdata, batch_size, input_profile_names,
                              actuator_names, target_profile_names,
                              profile_lookback, actuator_lookback, lookahead,
                              predict_deltas)
steps_per_epoch = len(train_generator)
val_steps = len(val_generator)

In [ ]:
batch_ind = np.random.randint(len(train_generator))
sig = 'temp'
inputs, targets = train_generator[batch_ind]
shotnums = train_generator.cur_shotnum[:,max(profile_lookback, actuator_lookback)]
times = train_generator.cur_times[:,max(profile_lookback, actuator_lookback)]
predictions = model.predict_on_batch(inputs)

psi = np.linspace(0,1,profile_length)
full_pred, full_true, delta_pred, delta_true, baseline = batch_denorm(inputs,targets,predictions,param_dict,sig)

In [ ]:
fig, ax = plot_batch(delta_true,delta_pred,np.zeros_like(baseline),psi,['true','pred','baseline'],['psi','delta etemp'], shotnums,times)  

In [ ]:
fig, ax = plot_batch(full_true,full_pred,baseline,psi,['true','pred','baseline'],['psi','etemp'], shotnums,times)  